# 87651260 Applications of Geographic Information System

# Geolocation

## Attawut Nardkulpat, Reseach Officer, Burapha University
### attawut@buu.ac.th
### 14/08/2020

In [1]:
#!pip install geopandas
!pip install GeoText
!pip install geopy

In [3]:
import matplotlib as plt
%matplotlib inline
import pandas as pd
import matplotlib.patches as mpatches
#import plotly.express as px
from geotext import GeoText
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import requests
from bs4 import BeautifulSoup
import os
import re
from functools import reduce

In [4]:
df2 = pd.read_csv('2020-03-04 Coronavirus Tweets.CSV',encoding='utf-8')
df2_text = df2[["text"]]
df2_text.to_string()
df2_text1 = df2_text.reset_index(drop=True)
df2_text1['geotext'] = df2_text1["text"].apply(lambda text: GeoText(text))

In [5]:
df2_text1

,text,geotext
0,#CoronaVirusIndonesia \r\n#CoronaBukaBorokRezi...,<geotext.geotext.GeoText object at 0x000001E27...
1,#CoronaVirusIndonesia \r\n#CoronaBukaBorokRezi...,<geotext.geotext.GeoText object at 0x000001E27...
2,Chinaโ€s wildlife trade via @SCMPgraphics ht...,<geotext.geotext.GeoText object at 0x000001E27...
3,Proof that the media such as @CNN is unnecessa...,<geotext.geotext.GeoText object at 0x000001E27...
4,Indians are a shitty breed of humans. With the...,<geotext.geotext.GeoText object at 0x000001E27...
...,...,...
7675,@tribelaw @jaketapper @SenSchumer It gives the...,<geotext.geotext.GeoText object at 0x000001E27...
7676,Lol Wooks are high key mad at about corona vir...,<geotext.geotext.GeoText object at 0x000001E27...
7677,Henry says the woman was around a tour group a...,<geotext.geotext.GeoText object at 0x000001E27...
7678,Contributed to this story by @pg_sobota: State...,<geotext.geotext.GeoText object at 0x000001E27...


# Extract Location

In [6]:
a = []
for i in range(0,len(df2_text1)):
    a  += (df2_text1['geotext'][i].cities)
city_mentions = pd.DataFrame(a, columns=['city'])

In [7]:
city_mentions

,city
0,Hyderabad
1,Wuhan
2,Wuhan
3,Mumbai
4,Brussels
...,...
1285,Florida
1286,March
1287,Vancouver
1288,Ontario


In [8]:
rank = city_mentions['city'].value_counts(ascending = False)
rank

Corona         245
Wuhan          122
March           60
Obama           59
Seattle         38
              ... 
Anna             1
Tortona          1
Long Island      1
New Orleans      1
Atherton         1
Name: city, Length: 281, dtype: int64

In [9]:
cities_to_remove = ['Corona', 'March', 'Of','Most','Obama','Un','Una','Bo','Best','Ho','Nice','University','Man','Police','Como']

In [10]:
city_mentions = city_mentions[~city_mentions['city'].isin(cities_to_remove)]
city_mentions

,city
0,Hyderabad
1,Wuhan
2,Wuhan
3,Mumbai
4,Brussels
...,...
1284,Barrington
1285,Florida
1287,Vancouver
1288,Ontario


In [11]:
rank = city_mentions['city'].value_counts()
rank

Wuhan         122
Seattle        38
Tokyo          29
Delhi          28
Washington     24
             ... 
Seria           1
Florence        1
Anna            1
Tortona         1
Atherton        1
Name: city, Length: 267, dtype: int64

In [12]:
rank = pd.DataFrame(rank)
rank

,city
Wuhan,122
Seattle,38
Tokyo,29
Delhi,28
Washington,24
...,...
Seria,1
Florence,1
Anna,1
Tortona,1


In [13]:
rank.to_csv('test_rank.csv')

# Extract Geolocation

In [19]:
df = pd.read_csv("test_rank.csv",encoding='utf-8')
df

,Unnamed: 0,city
0,Wuhan,122
1,Seattle,38
2,Tokyo,29
3,Delhi,28
4,Washington,24
5,New York,24
6,London,24
7,Asia,23
8,Florida,18


In [20]:
df.columns = ['city','count']
df

,city,count
0,Wuhan,122
1,Seattle,38
2,Tokyo,29
3,Delhi,28
4,Washington,24
5,New York,24
6,London,24
7,Asia,23
8,Florida,18


In [21]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [22]:
#use geopy's geolocator to find information on all the cities in the list
chrome_user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.92 Safari/537.36"

geolocator = Nominatim(timeout=10,user_agent=chrome_user_agent)

lat_lon = []
for i in df.city: 
    try:
        location = geolocator.geocode(i)
        if location:
            lat_lon.append(location)
    except GeocoderTimedOut as e:
        print("Error: geocode failed on input %s with message %s"%
             (i, e))

In [23]:
lat_lon

[Location(武汉市, 江岸区, 武汉市, 湖北省, 430062, China 中国, (30.5951051, 114.2999353, 0.0)),
 Location(Seattle, King County, Washington, United States of America, (47.6038321, -122.3300624, 0.0)),
 Location(東京都, 日本 (Japan), (35.6828387, 139.7594549, 0.0)),
 Location(Delhi, Kotwali Tehsil, Central Delhi, Delhi, 110006, India, (28.6517178, 77.2219388, 0.0)),
 Location(Washington, District of Columbia, 20500, United States of America, (38.8949855, -77.0365708, 0.0)),
 Location(New York, United States of America, (40.7127281, -74.0060152, 0.0)),
 Location(London, Greater London, England, United Kingdom, (51.5073219, -0.1276474, 0.0)),
 Location(Asia, (51.2086975, 89.2343748, 0.0)),
 Location(Florida, United States of America, (27.7567667, -81.4639835, 0.0))]

In [24]:
#create a df for raw output from the geolocator, maybe put cleaning directly after this; or append to original df
city_data = pd.DataFrame(lat_lon, columns=['raw_data','raw_data2'])

#re-order the cols
city_data = city_data[['raw_data2', 'raw_data']]

#get the city names from raw_data
city_data['city'] = city_data['raw_data'].str.split(',').str[0]

In [25]:
city_data

,raw_data2,raw_data,city
0,"(30.5951051, 114.2999353)","武汉市, 江岸区, 武汉市, 湖北省, 430062, China 中国",武汉市
1,"(47.6038321, -122.3300624)","Seattle, King County, Washington, United State...",Seattle
2,"(35.6828387, 139.7594549)","東京都, 日本 (Japan)",東京都
3,"(28.6517178, 77.2219388)","Delhi, Kotwali Tehsil, Central Delhi, Delhi, 1...",Delhi
4,"(38.8949855, -77.0365708)","Washington, District of Columbia, 20500, Unite...",Washington
5,"(40.7127281, -74.0060152)","New York, United States of America",New York
6,"(51.5073219, -0.1276474)","London, Greater London, England, United Kingdom",London
7,"(51.2086975, 89.2343748)",Asia,Asia
8,"(27.7567667, -81.4639835)","Florida, United States of America",Florida


In [26]:
#change the coordinates to a string
city_data['raw_data2'] = city_data['raw_data2'].astype(str)

#split the coordinates using the comma as the delimiter
city_data[['lat','lon']] = city_data.raw_data2.str.split(",",expand=True,)

#remove the parentheses
city_data['lat'] = city_data['lat'].map(lambda x:x.lstrip('()'))
city_data['lon'] = city_data['lon'].map(lambda x:x.rstrip('()'))

#convert the lat and lon columns back to floats
city_data = city_data.astype({'lat': 'float64', 'lon': 'float64'})

#drop the raw data columns
city_data.drop(['raw_data2', 'raw_data'], axis = 1, inplace=True)

In [27]:
city_data

,city,lat,lon
0,武汉市,30.595105,114.299935
1,Seattle,47.603832,-122.330062
2,東京都,35.682839,139.759455
3,Delhi,28.651718,77.221939
4,Washington,38.894985,-77.036571
5,New York,40.712728,-74.006015
6,London,51.507322,-0.127647
7,Asia,51.208697,89.234375
8,Florida,27.756767,-81.463983


In [28]:
#merge the two dataframes
merged = pd.merge(df, city_data, left_index=True, right_index=True)

In [29]:
merged

,city_x,count,city_y,lat,lon
0,Wuhan,122,武汉市,30.595105,114.299935
1,Seattle,38,Seattle,47.603832,-122.330062
2,Tokyo,29,東京都,35.682839,139.759455
3,Delhi,28,Delhi,28.651718,77.221939
4,Washington,24,Washington,38.894985,-77.036571
5,New York,24,New York,40.712728,-74.006015
6,London,24,London,51.507322,-0.127647
7,Asia,23,Asia,51.208697,89.234375
8,Florida,18,Florida,27.756767,-81.463983


In [30]:
merged.to_csv("geolocation_test.csv")